In [1]:
import mxnet as mx

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Anaconda3\lib\site-packages\urllib3\contrib\pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
x = mx.nd.array([[1, 2, 3, 4], [2, 4, 5, 6]])

In [3]:
x.attach_grad()

In [6]:
with mx.autograd.record():
    y = 2 * x + x ** 2

In [9]:
y.backward()

In [10]:
x.grad


[[ 4.  6.  8. 10.]
 [ 6. 10. 12. 14.]]
<NDArray 2x4 @cpu(0)>